# Shor's Algorithm

**Shor's Algorithm** is an quantum kata designed to teach the basics of Shor's Algoirithm, which efficiently factorises large numbers on a quantum computer.

Each task is wrapped in one operation preceded by the description of the task. Your goal is to fill in the blank (marked with the // ... comments) with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; harder ones are marked with asterisks.

## Part 1: A Classical Implementation

Initially, we will implement the classical framework around the algorithm, and a classical version of the implimentation. 

First, we need to check some special cases for the factorising algorithm.

### Task 1.1. IsEven

This one use used to check if we can return the very fast calculation of $(2, N/2)$ as the factorisation.

#### Input:
Any positive integer $N$
#### Goal:
Return true if the number is even

In [20]:
%kata IsEven_Test

function IsEven(N : Int) : Bool {
    // ...
}

Starting IsEven_Test...


IsEven failed on 2
	Expected:	True
	Actual:	False
Try again!


### Task 1.2. IsPrime

Used to check we can actually factor the number we are given. This is useful to prevent infinite loops and errors later on in the code.

This could be done by finding the factors, but that defeats the purpose of Shor's Algorithm. Faster methods (i.e. polynomial with respect to the size of $N$) can be found [here](https://en.wikipedia.org/wiki/Primality_test#Example_code).

#### Input:
A positive integer $N$
#### Goal:
Return true if $N$ is prime.

In [21]:
%kata IsPrime_Test

function IsPrime(N : Int) : Bool {
    // ...
}

/snippet_.qs(2,1): error QS6307: Not all code paths return a value.


With these two tests, we can confirm that N is a product of two distinct prime numbers, so $N = pq$. 

### Task 1.3. Classical order finding

Shor's Algorithm uses a method called order finding for factorising. The order of two numbers $a, N$ is the value of $r > 0$ such that $a^r\mod N = 1$. Consider the example of $a=2$, $N=15$.

| r | $$a^r$$ | $$a^r \mod N$$ |
| - | ----- | ----------- |
| 0 | 1     | 1           |
| 1 | 2     | 2           |
| 2 | 4     | 4           |
| 3 | 8     | 8           |
| 4 | 16    | 1           |
| 5 | 32    | 2           |
| ... | ... | ...         |

So, the order of $a, N$ is $4$. We can apply this concept to any two relatively prime numbers (where the greatest common divisor is 1). 

Classically the order cannot 

#### Input:
Two positive, relatively prime ingeters $a$ and $N$ such that 

In [3]:
operation FindOrderClassicalHelper(a : BigInt, N: BigInt) : Int {
    mutable power = 0;
    repeat {
        set power += 1;
    } until (a^power % N == 1L);
    return power;
}

FindOrderClassicalHelper

In [4]:
%kata FindOrderClassical_Test

open Microsoft.Quantum.Convert;

operation FindOrderClassical(a : Int, N : Int) : Int{
    return FindOrderClassicalHelper(IntAsBigInt(a),IntAsBigInt(N));
}

Starting FindOrderClassical_Test...


Success!

### 1.4 Generate random number

In [5]:
%kata GenerateRandomNumber_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation GenerateRandomNumber(N : Int) : Int {
    use register = Qubit[BitSizeI(N-2)];
    mutable result = 0;
    repeat {
        ApplyToEachCA(H,register);
        set result = MeasureInteger(LittleEndian(register));
    } until ((result + 2) < N);
    return result+2;
}

Starting GenerateRandomNumber_Test...


Success!

### 1.5 General Case

In [7]:
%kata GeneralCase_Test

operation GeneralCase(OrderFinder : ((Int, Int)=>Int), N : Int) : (Int, Int) {
    mutable result = 0;
    repeat {
        let a = GenerateRandomNumber(N);
        let gcd = GreatestCommonDivisorI(a,N);
        if (gcd > 1) {
            return (gcd, N/gcd);
        }
        let r = OrderFinder(a,N);
        if (IsEven(r)) {
            let x = (a^(r/2) - 1) % N;
            let gcdX = GreatestCommonDivisorI(x,N);
            if (gcdX > 1) {
                set result = gcdX;
            }
        }
    } until (result != 0);
    return (result, N/result);
}

Starting GeneralCase_Test...


Success!

### 1.6 Full Shor's Implimentation

In [8]:
operation ShorsAlgorithm(OrderFinder : (Int, Int)=> Int, N : Int) : (Int, Int) {
    if IsEven(N) {
        return (2, N/2);
    }
    if IsPrime(N) {
        fail IntAsString(N) + " is prime, so doesn't have factors.";
    }
    return GeneralCase(OrderFinder, N);
}

ShorsAlgorithm

### 1.7 Classical Shor's Implimentation

In [9]:
%kata ShorsAlgorithmClassical_Test

operation ShorsAlgorithmClassical(N : Int) : (Int, Int) {
    return ShorsAlgorithm(FindOrderClassical, N);
}

Starting ShorsAlgorithmClassical_Test...


Success!

In [10]:
%simulate ShorsAlgorithmClassical N=15

(3, 5)

## Part 2: Quantum Implementation of Order Finding
### 2.1 Oracle

In [11]:
%kata OrderFindingOracle_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation OrderFindingOracle(a : Int, N : Int, power : Int, target : Qubit[]) : Unit is Adj+Ctl {
    MultiplyByModularInteger(ExpModI(a, power, N), N, LittleEndian(target));
}

Starting OrderFindingOracle_Test...


Success!

In [12]:
%kata PrepareEigenstate_Test

operation PrepareEigenstate(eigenstate : Qubit[]) : Unit is Adj+Ctl {
    X(eigenstate[0]);
}

Starting PrepareEigenstate_Test...


Success!

In [13]:
%kata ApplyQuantumPhaseEstimation_Test

open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Characterization;

operation ApplyQuantumPhaseEstimation(oracle : DiscreteOracle, eigenstate : Qubit[], result : Qubit[]) : Int {
    let resultBELE = LittleEndianAsBigEndian(LittleEndian(result));
    QuantumPhaseEstimation(oracle,eigenstate,resultBELE);
    return MeasureInteger(LittleEndian(result));
}

Starting ApplyQuantumPhaseEstimation_Test...
Did not find valid result on attempt 1, retrying...


Success!

In [14]:
%kata PhaseResultToPeriod_Test

open Microsoft.Quantum.Math;

operation PhaseResultToPeriod(phaseResult : Int, bitsPrecision : Int, N : Int) : Int {
    let fractionResult = Fraction(phaseResult,2^bitsPrecision);
    let simplifiedFraction = ContinuedFractionConvergentI(fractionResult, N);
    let (numerator, period) = simplifiedFraction!;
    return AbsI(period);
}

Starting PhaseResultToPeriod_Test...


Success!

In [15]:
%kata FindOrderQuantum_Test

open Microsoft.Quantum.Diagnostics;

operation FindOrderQuantum(a : Int, N : Int) : Int {
    mutable result = 1;
    let bitsize = BitSizeI(N);
    let bitsPrecision = 2 * bitsize + 1; // decide on precision?
    
    repeat {        
        // setup register holding 'eigenstate' of |1>
        use eigenstate = Qubit[bitsize];
        PrepareEigenstate(eigenstate);
        
        // setup register to contain QFE result
        use phaseResult = Qubit[bitsPrecision];
        
        // measure the result from QPE
        let oracle = DiscreteOracle(OrderFindingOracle(a,N,_,_));
        //QuantumPhaseEstimation(oracle, eigenstate, LittleEndianAsBigEndian(phaseResultLE));
        let phaseResultI = ApplyQuantumPhaseEstimation(oracle, eigenstate, phaseResult);
        
        ResetAll(eigenstate);
        
        
        // calculate the period based of the phase result
        let period = PhaseResultToPeriod(phaseResultI,bitsPrecision,N);
        
        // deal with a zero return value
        if (period == 0) { set result = 1; }
        // account for sub factors
        else { set result = (period * result) / GreatestCommonDivisorI(result, period); }
        
    }
    until (ExpModI(a,result,N) == 1);
    return result;
}

Starting FindOrderQuantum_Test...


Success!

In [16]:
%kata ShorsAlgorithmQuantum_Test

operation ShorsAlgorithmQuantum(N : Int) : (Int, Int) {
    return ShorsAlgorithm(FindOrderQuantum,N);
}

Starting ShorsAlgorithmQuantum_Test...


Success!

In [17]:
%simulate ShorsAlgorithmQuantum N=15

(3, 5)